In [1]:
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import collections as col
import re
import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
#from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
#from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import auc,roc_curve
import tensorflow as tf

In [26]:
df_train_feat = pd.DataFrame.from_csv('Model/train_feat_encoded.csv')
df_train_y = pd.DataFrame.from_csv('Model/train_y.csv')

In [27]:
train_feat_ar= df_train_feat.as_matrix()
train_y_ar= df_train_y.as_matrix()

In [19]:
x_i = tf.placeholder("float", [None, 215])
y_gold = tf.placeholder("float", [None, 1])

d_in = 215
d_hidden1 = 500
d_hidden2 = 500
d_hidden3 = 500
d_out = 1

W1 = tf.Variable(tf.random_normal([d_in, d_hidden1], mean= 0.01, stddev= 0.01))
b1 = tf.Variable(tf.random_normal([d_hidden1], mean= 0.01, stddev= 0.01))
W2 = tf.Variable(tf.random_normal([d_hidden1, d_hidden2], mean= 0.01, stddev= 0.01))
b2 = tf.Variable(tf.random_normal([d_hidden2], mean= 0.01, stddev= 0.01))
W3 = tf.Variable(tf.random_normal([d_hidden2, d_hidden3], mean= 0.01, stddev= 0.01))
b3 = tf.Variable(tf.random_normal([d_hidden3], mean= 0.01, stddev= 0.01))
W4 = tf.Variable(tf.random_normal([d_hidden3, d_out], mean= 0.01, stddev= 0.01))
b4 = tf.Variable(tf.random_normal([d_out], mean= 0.01, stddev= 0.01))

a1_i = tf.matmul(x_i, W1)+ b1
z1_i = tf.sigmoid(a1_i)
a2_i = tf.matmul(z1_i, W2)+ b2
z2_i = tf.sigmoid(a2_i)
a3_i = tf.matmul(z2_i, W3)+ b3
z3_i = tf.sigmoid(a3_i)
a4_i = tf.matmul(z3_i, W4)+ b4
y_hat = tf.sigmoid(a4_i)

loss = tf.pow((y_hat- y_gold), 2)
#cross_entropy = -tf.reduce_sum(y_gold* tf.log(tf.clip_by_value(P_ij, 1e-10, 1.0)))
optimiser = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

prediction = tf.round(y_hat)
mistakes = tf.not_equal(y_gold, prediction)
accuracy = 1- tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [ ]:
# Train and test
batch_sz= 30
iter_= int(Xi.shape[0]/ batch_sz)
epoch= 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load_model()
    
    init_dict= {x_i: Xi[:10000],
                y_gold: P_target_r[:10000]}
    print('Init train accuracy:', (sess.run(accuracy, feed_dict= init_dict)))
    
    test_dict= {x_i: val_Xi[10000:20000],
                y_gold: val_P_target_r[10000:20000]} 
    print('Init test accuracy:', (sess.run(accuracy, feed_dict= test_dict)))
    
    for e in range(epoch):
        e_loss= 0
        
        for i in range(iter_):
            i_loss= 0
            sta= i* batch_sz
            end= (i+ 1)* batch_sz
            iter_dict= {x_i: Xi[sta: end],
                        y_gold: P_target_r[sta: end]}
            sess.run(optimiser, feed_dict= iter_dict)
            e_loss+= sess.run(cross_entropy, feed_dict= iter_dict)
        
        if e% (epoch/ 10)== 0:
            print('Epoch', e, 'loss:', e_loss)
            print('Epoch train/test accur:', (sess.run(accuracy, feed_dict= init_dict)), (sess.run(accuracy, feed_dict= test_dict)))
            
        if (sess.run(accuracy, feed_dict= test_dict)) > 0.4:
            save_model(sess)
        
    print('Final train accuracy:', (sess.run(accuracy, feed_dict= init_dict)))
    print('Final test accuracy:', (sess.run(accuracy, feed_dict= test_dict)))
    
    # TensorFlow save model
    save_model(sess)